<a href="https://colab.research.google.com/github/jatoogunhyo/sec_edgar/blob/main/EDGAR_Tools_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [(링크) EDGARTOOLS 라이브러리 사용안내](https://github.com/dgunning/edgartools)

# 필요한 환경 설정

In [ ]:
!pip install edgartools
from edgar import *
import re
import pandas as pd
import time
from tqdm.notebook import tqdm  # 구글 코랩에 적합한 tqdm 모듈
import requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.5 MB/s eta 0:00:00


# SEC 서식
(배경) SEC에 제출해야 하는 공시는 다음과 같음

| 서식 유형                | 호출 코드 | 서식명                | 설명                                                                 |
|--------------------------|-----------|-----------------------|----------------------------------------------------------------------|
| **재무 보고서**          | 10-K      | 연례 보고서           | 기업의 연간 재무 상태와 운영 성과를 상세히 보고하는 문서입니다.         |
|                          | 10-Q      | 분기 보고서           | 기업의 분기별 재무 상태와 운영 성과를 보고하는 문서입니다.             |
|                          | 20-F      | Form 20-F            | 외국 기업이 SEC에 제출하는 연례 보고서입니다.                         |
|                          | 40-F      | Form 40-F            | 캐나다 기업이 SEC에 제출하는 연례 보고서입니다.                       |
| **중요 사건 보고서**      | 8-K       | Current Report           | 기업의 중요한 사건이나 변경 사항을 즉시 보고하는 문서입니다.          |
| **소유권 및 투자 보고서** | SC 13D    | Schedule 13D         | 5% 이상 주식을 보유한 투자자가 제출하는 문서로, 보유 목적과 계획을 상세히 기술합니다. |
|                          | SC 13G    | Schedule 13G         | 5% 이상 주식을 보유한 수동적 투자자가 제출하는 간소화된 보고서입니다. |
|                          | 13F-HR    | Form 13F             | 기관투자자가 보유한 주식 목록을 분기별로 보고하는 문서입니다.         |
| **내부자 거래 보고서**    | 3         | Form 3               | 기업의 임원, 이사, 또는 주요 주주가 된 개인이 자신의 소유권을 처음으로 보고하는 문서입니다. |
|                          | 4         | Form 4               | 기업 내부자의 주식 거래를 보고하는 문서입니다.                       |
|                          | 5         | Form 5               | 보고되지 않은 내부자 거래를 연간으로 보고하는 문서입니다.             |
| **증권 등록**            | S-1       | Form S-1             | 기업이 IPO를 통해 주식을 공개적으로 발행할 때 사용하는 등록 문서입니다. |
|                          | S-3       | Form S-3             | 이미 상장된 기업이 추가 증권을 발행할 때 사용하는 간소화된 등록 문서입니다. |
|                          | S-4       | Form S-4             | 기업 합병이나 인수 시 사용하는 등록 문서입니다.                       |
|                          | S-8       | Form S-8             | 직원 스톡 옵션 계획 등을 위해 증권을 등록할 때 사용하는 문서입니다.   |
|                          | F-1       | Form F-1             | 외국 기업이 미국에서 증권을 발행할 때 사용하는 등록 문서입니다.       |
|                          | F-3       | Form F-3             | 외국 상장 기업이 추가 증권을 발행할 때 사용하는 간소화된 등록 문서입니다. |
|                          | F-4       | Form F-4             | 외국 기업의 합병이나 인수 시 사용하는 등록 문서입니다.               |
|                          | F-6       | Form F-6             | 미국 주식 예탁 증서(ADR)를 등록할 때 사용하는 문서입니다.             |
| **기타 등록 및 공시**     | D         | Form D               | 증권의 면제를 신청할 때 사용하는 문서로, 주로 사모펀드에서 사용됩니다. |
|                          | 144       | Form 144             | 제한된 증권의 매각을 계획할 때 사용하는 통지서입니다.                 |
|                          | EFFECT    | Notice of Effectiveness | 증권 등록이 SEC에 의해 효력을 얻었음을 알리는 통지서입니다.        |
| **크라우드펀딩 보고서**   | C         | Form C               | 크라우드펀딩을 통해 자금을 모집하는 기업이 사용하는 문서입니다.       |
|                          | C-U       | Form C-U             | 크라우드펀딩 진행 상황을 업데이트하는 문서입니다.                     |
|                          | C-AR      | Form C-AR            | 크라우드펀딩 완료 후 연례 보고서입니다.                              |
|                          | C-TR      | Form C-TR            | 크라우드펀딩 종료 보고서입니다.                                      |
| **포트폴리오 보고서**     | NPORT-P   | Form NPORT-P         | 투자 회사의 월간 포트폴리오 보고서입니다.                           |
| **기타**                 | MA-I      | Form MA-I            | Municipal Advisor의 초기 등록 시 사용하는 문서입니다.               |


# 서식별 문서 확인하기

## 확인할 공시서식 지정하기
* form 파라미터에 지정하면 됨

In [ ]:
# SEC API를 사용하려면 본인의 이메일을 전송해야함
set_identity("JaeKwang jaekwang.jung@krx.co.kr")

# 서식명, 연도를 사용해서 서식 불러오기
filings = get_filings(form=['8-K'], year=2024)
filings.to_pandas()

,form,company,cik,filing_date,accession_number
0,8-K,180 Life Sciences Corp.,1690080,2024-12-31,0001213900-24-113675
1,8-K,1847 Holdings LLC,1599407,2024-12-31,0001213900-24-114023
2,8-K,"ALLURION TECHNOLOGIES, INC.",1964979,2024-12-31,0001193125-24-287091
3,8-K,"APPLIED OPTOELECTRONICS, INC.",1158114,2024-12-31,0001683168-24-009068
4,8-K,ASPAC III Acquisition Corp.,1890361,2024-12-31,0001213900-24-113961
...,...,...,...,...,...
70024,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000007
70025,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000009
70026,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000011
70027,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000013


In [ ]:
# 하나를 예시로 보기
filing = filings[16]
filing

# 공시를 보고싶으면 Primary Document에 대당하는 URL을 클릭하면 됨

╭───────────────────────────────── BERKSHIRE HATHAWAY ENERGY CO [1081316] 8-K 📄 ─────────────────────────────────╮
│ ╭──────────────────────┬────────────╮                                                                           │
│ │ 0001081316-24-000029 │ 2024-12-31 │                                                                           │
│ ╰──────────────────────┴────────────╯                                                                           │
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮                  │
│ │ Links: 🏠 Homepage 📄 Primary Document 📜 Full Submission Text                             │                  │
│ ├────────────────────────────────────────────────────────────────────────────────────────────┤                  │
│ │ 🏠 https://sec.gov/Archives/edgar/data/1081316/0001081316-24-000029-index.html             │                  │
│ │ 📄 https://sec.gov/Archives/edgar/data/1081316/000108131624000029/bhe-2024